In [1]:
import django 
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from sch.models import *
import datetime as dt
from django.db.models import CharField
from sch.actions import *
from sch.xviews import week

In [2]:
from sch.actions import *
from sch.xviews import week

In [3]:
week.get_days("2022-W42")

<WorkdayManager [<Workday: 2022 10 16>, <Workday: 2022 10 17>, <Workday: 2022 10 18>, <Workday: 2022 10 19>, <Workday: 2022 10 20>, <Workday: 2022 10 21>, <Workday: 2022 10 22>]>

In [4]:
emps = Employee.objects.in_other_slot(workday=Workday.objects.get(slug='2022-10-31'),shift=Shift.objects.get(name='7C'))
emps = emps.filter(shifts_trained=Shift.objects.get(name='7C'))
emps

<EmployeeManager [<Employee: Josh>, <Employee: Brianna>, <Employee: Danica>, <Employee: David>, <Employee: Nicki>, <Employee: Cheryl>]>

In [5]:
shiftsPerWeek = 0
for sft in Shift.objects.filter(start__hour__gte=10, cls='CPhT').values_list('occur_days',flat=True):
    shiftsPerWeek += len(sft)
shiftsPerWeek

28

In [22]:
NIGHT_PERCENTS = []
for emp in Employee.objects.all():
    if Slot.objects.filter(employee=emp).exists():
        perc = round(Slot.objects.filter(employee=emp,shift__start__hour__gte=10).count()/Slot.objects.filter(employee=emp).count()*100, 1)
        NIGHT_PERCENTS += [f"{emp}: {perc}% EVENINGS  {int(perc/4) *'+'}"]
        
NIGHT_PERCENTS
        

['Josh: 13.6% EVENINGS  +++',
 'Sabrina: 27.0% EVENINGS  ++++++',
 'Brianna: 0.0% EVENINGS  ',
 'Molly: 17.7% EVENINGS  ++++',
 'Doug: 30.4% EVENINGS  +++++++',
 'Brittanie: 0.0% EVENINGS  ',
 'Esperanza: 4.5% EVENINGS  +',
 'Carrie: 34.4% EVENINGS  ++++++++',
 'Elisia: 77.6% EVENINGS  +++++++++++++++++++',
 'Jaden: 22.7% EVENINGS  +++++',
 'Lynlee: 30.8% EVENINGS  +++++++',
 'Carol: 46.9% EVENINGS  +++++++++++',
 'Trisha: 25.7% EVENINGS  ++++++',
 'Danica: 100.0% EVENINGS  +++++++++++++++++++++++++',
 'Tiffany: 24.6% EVENINGS  ++++++',
 'Brianna S: 0.0% EVENINGS  ',
 'Amanda: 100.0% EVENINGS  +++++++++++++++++++++++++',
 'Michael: 100.0% EVENINGS  +++++++++++++++++++++++++',
 'Mona: 100.0% EVENINGS  +++++++++++++++++++++++++',
 'Mary: 100.0% EVENINGS  +++++++++++++++++++++++++',
 'Mark B: 100.0% EVENINGS  +++++++++++++++++++++++++',
 'Andrew: 51.6% EVENINGS  ++++++++++++',
 'Mei: 100.0% EVENINGS  +++++++++++++++++++++++++',
 'Tony: 100.0% EVENINGS  +++++++++++++++++++++++++',
 'Sam: 4

In [8]:
today = Workday.objects.get(date=dt.date.today())
today_slots = Slot.objects.filter(workday=today).order_by('shift__start')
for slot in today_slots:
    if slot.employee.cls == "CPhT":
        print (slot)

221109-MI Jaden
221109-7P Esperanza
221109-7C Trisha
221109-S Brianna
221109-OP Brittanie
221109-EI Danica
221109-EP Elisia
221109-3 Mona
221109-N Amanda


In [9]:
weekWds = today.siblings_iweek
weekSlots = Slot.objects.filter(workday__in=weekWds).order_by('-employee')
for slot in weekSlots[:20]:
    print(slot)

221106-RS Corwin
221106-EP Cheryl
221107-EP Cheryl
221108-EP Cheryl
221110-EP Cheryl
221108-RS Erin
221109-RS Erin
221111-RS Erin
221106-C Kat
221110-PC Kat
221111-M Kat
221111-MI Leslie
221107-M Alex
221108-E Alex
221110-E Alex
221107-RS Shaun
221108-I Shaun
221109-PC Shaun
221112-RS Shaun
221108-PC Kristen


In [10]:
def getTradableSlots(slot):
    print('INPUT: ', slot)
    emp = slot.employee
    sft = slot.shift
    wd = slot.workday
    weekdays = slot.workday.siblings_iweek
    daysOn = Slot.objects.filter(workday__in = weekdays,employee = emp).values('workday').distinct()
    daysOff = weekdays.exclude(pk__in=daysOn)
    print('daysOff:',list(daysOff.values_list('date__day',flat=True)))
    tenable = []
    slotDaysOff = Slot.objects.filter(workday__in=daysOff, shift__in=emp.shifts_available.all()).order_by('workday')
    print(slotDaysOff)
    for slotB in slotDaysOff:
        if emp in slot.fillableBy():
            if slotB.employee in slot.fillableBy():
                tenable.append(slotB)
    print('tenableTrades:',len(tenable),[f'{i.shift}>{i.workday.date.day}' for i in tenable])
    return tenable



In [11]:
slots = Slot.objects.filter(employee__name="Trisha",workday__iweek=43,workday__date__year=2022)
for slot in slots:
    print('-------------')
    print(slot.is_turnaround)
    getTradableSlots(slot)

-------------
0
INPUT:  221025-S Trisha
daysOff: [23, 26, 28, 29]
<SlotManager [<Slot: 221023-7C Josh>, <Slot: 221023-7P Esperanza>, <Slot: 221023-EP David>, <Slot: 221023-3 Mona>, <Slot: 221023-N Amanda>, <Slot: 221026-MI Brianna>, <Slot: 221026-S Jaden>, <Slot: 221026-7C Nicki>, <Slot: 221026-7P Josh>, <Slot: 221026-OP Brittanie>, <Slot: 221026-EI Danica>, <Slot: 221026-EP Elisia>, <Slot: 221026-3 Mona>, <Slot: 221026-N Amanda>, <Slot: 221028-MI Sabrina>, <Slot: 221028-S Esperanza>, <Slot: 221028-7C Cheryl>, <Slot: 221028-7P Brianna>, <Slot: 221028-OP Brittanie>, <Slot: 221028-EI Tiffany>, '...(remaining elements truncated)...']>
tenableTrades: 13 ['7C>23', '7P>23', 'MI>26', 'S>26', '7P>26', 'EI>26', 'S>28', '7P>28', 'EI>28', '3>28', '7C>29', '7P>29', '3>29']
-------------
0
INPUT:  221027-7P Trisha
daysOff: [23, 26, 28, 29]
<SlotManager [<Slot: 221023-7C Josh>, <Slot: 221023-7P Esperanza>, <Slot: 221023-EP David>, <Slot: 221023-3 Mona>, <Slot: 221023-N Amanda>, <Slot: 221026-MI Bria

In [12]:
wds = Workday.objects.filter(date__year=2022,ischedule=7)
ss = Slot.objects.filter(workday__in=wds)
ss

<SlotManager [<Slot: 221016-MI Tiffany>, <Slot: 221016-7P Brianna>, <Slot: 221016-C Carrie>, <Slot: 221016-EI Jaden>, <Slot: 221016-EP Elisia>, <Slot: 221016-3 Danica>, <Slot: 221016-R3 Andrew>, <Slot: 221016-RN Mary>, <Slot: 221016-7C David>, <Slot: 221016-RS Shaun>, <Slot: 221017-7P Brianna>, <Slot: 221017-OP Brittanie>, <Slot: 221017-MO Brianna S>, <Slot: 221017-3 Danica>, <Slot: 221017-RN Mary>, <Slot: 221017-S Trisha>, <Slot: 221017-I Molly>, <Slot: 221017-EI Nicki>, <Slot: 221017-EP Cheryl>, <Slot: 221017-E Tony>, '...(remaining elements truncated)...']>

In [13]:
for slot in Slot.objects.all():
    slot.save()

In [14]:
for wd in Workday.objects.filter(iweek=44):
    print(wd,wd.week)

2022 10 30 None
2022 10 31 None
2022 11 01 None
2022 11 02 None
2022 11 03 None
2022 11 04 None
2022 11 05 None


In [15]:
Workday.objects.all().count()

366

In [16]:
from django import forms
import datetime as dt
class ArticleForm(forms.Form):
     title = forms.CharField()
     pub_date = forms.DateField()

In [23]:
""" 
Janet’s ducks lay 16 eggs per day. She eats three for breakfast 
every morning and bakes muffins for her friends every day with 
four. She sells the remainder for $2 per egg. How much does she
make every day? SHOW YOUR CALCULATIONS
"""

# Define the number of eggs Janet's ducks lay per day.
eggs_per_day = 16

# Define the number of eggs Janet eats for breakfast every morning.
eggs_for_breakfast = 3

# Define the number of eggs Janet uses to bake muffins for her friends every day.
eggs_for_muffins = 4

# Define the price Janet sells her eggs for.
eggs_price = 2

# Calculate the number of eggs Janet has left over after eating breakfast and baking muffins.
eggs_leftover = eggs_per_day - eggs_for_breakfast - eggs_for_muffins

# Calculate the amount of money Janet makes selling her eggs.
eggs_money = eggs_leftover * eggs_price

# Print the number of eggs Janet has left over after eating breakfast and baking muffins.
print("Janet has " + str(eggs_leftover) + " eggs left over after eating breakfast and baking muffins.")

# Print the amount of money Janet makes selling her eggs.
print("Janet makes $" + str(eggs_money) + " selling her eggs.")

Janet has 9 eggs left over after eating breakfast and baking muffins.
Janet makes $18 selling her eggs.
